In [10]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

# Turn inFile into a one hot label
def createOneHot(inFile,amountOfClasses):
    outFile = np.zeros((inFile.shape[0],inFile.shape[1], amountOfClasses));
    for i in range(0,inFile.shape[0]):
        for j in range(0,inFile.shape[1]):
            #if inFile[i,j] != 0:
            outFile[i,j,int(inFile[i,j]-1)] = 1;
    return outFile;

# Turn 41 orthomosaics into one array. From 2 separate paths, 1-25 and 1-16
def loadOneOrthomosaic(imagePath1,imagePath2):
    temp= imageio.imread(imagePath1 + '1.png');
    inputData = np.zeros( (temp.shape[0],temp.shape[1],inputChannels) ,dtype=np.dtype('uint8'))
    for i in range(1,25):
        temp= imageio.imread(imagePath1 + str(i) + '.png');
        inputData[:,:,i] = temp[:,:,0];
    for i in range(1,16):
        temp= imageio.imread(imagePath2 + str(i) + '.png');
        inputData[:,:,i+25] = temp[:,:,0];
    return np.expand_dims(inputData,axis=0)

# Turn 41 orthomosaics into one array. From 1 path with Bands 1 to 41
def loadOneOrthomosaic(imagePath):
    temp= imageio.imread(imagePath + '1.png');
    inputData = np.zeros(( temp.shape[0], temp.shape[1], inputChannels),dtype=np.dtype('uint8'))
    for i in range(1,41):
        temp= imageio.imread(imagePath + str(i) + '.png');
        inputData[:,:,i] = temp[:,:];
    return np.expand_dims( inputData, axis=0)

# Load one label from a .mat
def loadOneLabel(imagePath):
    temp = sio.loadmat(imagePath);
    trainOneHotLabels =  createOneHot( temp['labeledPicture'], amountOfClasses);
    return np.expand_dims( trainOneHotLabels, axis=0)

# Read training data and labels for an array of path tuples, returns (data,labels)
def readSet(paths):
    numberOfImages = len(paths);
    data = np.zeros( (numberOfImages,imageSize[1],imageSize[0],inputChannels), dtype=np.dtype('uint8'));
    labels = np.zeros((numberOfImages,imageSize[1],imageSize[0],amountOfClasses), dtype=np.dtype('uint8'))
    for i in range(0,numberOfImages):
        data[i,:,:,:] = loadOneOrthomosaic(paths[i][0]);
        labels[i,:,:,:] = loadOneLabel(paths[i][1]);
    return data,labels

def oneHotToCategorical(oneHotLabel):
    temp = np.zeros((oneHotLabel.shape[1],oneHotLabel.shape[2]), dtype=np.dtype('uint8') );
    for i in range(0,oneHotLabel.shape[1]):
        for j in range(0,oneHotLabel.shape[2]):
            #temp[i,j] = np.argmax(oneHotLabel[0,i,j,:]) + 1;
            temp[i,j] = np.argmax(oneHotLabel[0,i,j,:]);
    return temp

def predictAndSaveOneImage(inPath,outPath,FCNmodel):
    testImage = loadOneOrthomosaic(inPath);
    testLabelPredict = FCNmodel.predict(testImage);
    testLabelPredictCat = oneHotToCategorical(testLabelPredict);
    sio.savemat(outPath, mdict={'testLabelPredict': testLabelPredictCat});

def predictAndSaveSet(pathArray, FCNmodel):
    numberOfImages = len(pathArray);
    for i in range(0,numberOfImages):
        predictAndSaveOneImage(pathArray[i][0], pathArray[i][1], FCNmodel)
        print('Saved ' + pathArray[i][1])
        
def plotImage(image):
    plt.imshow(image)

In [3]:
# Data paths
basePath = '/Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/';

imagePath1 = basePath + 'Data/FIP/20170622/Band';
labelpath1 = basePath + 'Labels/FIP20170622.mat';
imagePath2 = basePath + 'Data/Ximea_Tamron/20170510/Band';
labelpath2 = basePath + 'Labels/XimeaT20170510.mat';
imagePath3 = basePath + 'Data/Ximea_Tamron/20170622/Band';
labelpath3 = basePath + 'Labels/XimeaT20170622.mat';

trainPaths = [[imagePath1,labelpath1],[imagePath2,labelpath2],[imagePath3,labelpath3]];
del imagePath1,labelpath1,imagePath2,labelpath2,imagePath3,labelpath3

inPath1 = basePath + 'testData/Ximea_Tamron/20170613/Band';
outpath1 = basePath + 'testLabelPredict/XimeaT_20170613.mat';
inPath2 = basePath + 'testData/FIP/20170531/Band';
outpath2 = basePath + 'testLabelPredict/FIP_20170531.mat';
inPath3 = basePath + 'testData/FIP/20170622/Band';
outpath3 = basePath + 'testLabelPredict/FIP_20170622.mat';
inPath4 = basePath + 'testData/FIP/20170802/Band';
outpath4 = basePath + 'testLabelPredict/FIP_20170802.mat';
inPath5 = trainPaths[0][0];
outpath5 = basePath + 'testLabelPredict/FIP_20170622TRAIN.mat';
inPath6 = trainPaths[1][0];
outpath6 = basePath + 'testLabelPredict/XimeaT_20170510TRAIN.mat';
inPath7 = trainPaths[2][0];
outpath7 = basePath + 'testLabelPredict/XimeaT_20170622TRAIN.mat';

testTrainPaths = [[inPath5,outpath5],[inPath6,outpath6],[inPath7,outpath7]];
testPaths = [[inPath1,outpath1],[inPath2,outpath2],[inPath3,outpath3],[inPath4,outpath4]];
del inPath1,outpath1,inPath2,outpath2,inPath3,outpath3,inPath4,outpath4,inPath5,outpath5,inPath6,outpath6,inPath7,outpath7,basePath

# Data parameters
amountOfClasses = 9;
inputChannels = 41;
imageSize = [1600,1600]; # [X,Y]

In [13]:
modelBasicNN = Sequential([
    Conv2D(20, (1,1), input_shape=(imageSize[0], imageSize[1], inputChannels), activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])
modelBasicNN.summary()
modelBasicNN.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy']);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 1600, 1600, 20)    840       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 1600, 1600, 9)     189       
Total params: 1,029
Trainable params: 1,029
Non-trainable params: 0
_________________________________________________________________


In [3]:
model = Sequential([
    Conv2D(20, 5, input_shape=(imageSize[0], imageSize[1], inputChannels), activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(20, 6, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])
model.summary()
model.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy']);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1596, 1596, 20)    20520     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 798, 798, 20)      0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 1600, 1600, 20)    14420     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1600, 1600, 8)     168       
Total params: 35,108
Trainable params: 35,108
Non-trainable params: 0
_________________________________________________________________


In [111]:
from keras.models import *
from keras.layers import Conv2D, MaxPooling2D, Input, Cropping2D, UpSampling2D,concatenate

#crop1 = Cropping2D(cropping=((0, 0), (0, 0)))(conv1) # cropping=((Top, Bottom), (Left, Right))

model_inputs = Input(shape=(imageSize[0], imageSize[1], inputChannels))

conv1 = Conv2D(64,3, activation='relu', padding='same')(model_inputs)
conv1 = Conv2D(64,3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')(conv1)

conv2 = Conv2D(128,3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128,3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')(conv2)

conv3 = Conv2D(256,3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256,3, activation='relu', padding='same')(conv3)

upconv4 = Conv2D(128,2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv3))
concat4 = concatenate([upconv4,conv2],axis=3)
conv4 = Conv2D(128,3, activation='relu', padding='same')(concat4)
conv4 = Conv2D(128,3, activation='relu', padding='same')(conv4)

upconv5 = Conv2D(64,2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv4))
concat5 = concatenate([upconv5,conv1],axis=3)
conv5 = Conv2D(64,3, activation='relu', padding='same')(concat5)
conv5 = Conv2D(64,3, activation='relu', padding='same')(conv5)

conv6 = Conv2D(amountOfClasses, (1,1), activation='softmax', padding='valid')(conv5)


UNetModel = Model(inputs = model_inputs, outputs = conv6)
UNetModel.compile(optimizer='adagrad',
              loss='categorical_crossentropy',
              metrics=['accuracy']);
print(UNetModel.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 1600, 1600, 4 0                                            
__________________________________________________________________________________________________
conv2d_146 (Conv2D)             (None, 1600, 1600, 6 23680       input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_147 (Conv2D)             (None, 1600, 1600, 6 36928       conv2d_146[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_36 (MaxPooling2D) (None, 800, 800, 64) 0           conv2d_147[0][0]                 
__________________________________________________________________________________________________
conv2d_148

In [5]:
# Load the training data and labels
trainData,trainLabels = readSet(trainPaths)

In [14]:
# Train model
modelBasicNN.fit(trainData, trainLabels, epochs=3);

Epoch 1/3
3/3 [==============================] - 38s 13s/step - loss: 2.8838 - acc: 0.1562
Epoch 2/3
3/3 [==============================] - 31s 10s/step - loss: 3.9083 - acc: 0.7788
Epoch 3/3
3/3 [==============================] - 32s 11s/step - loss: 2.8042 - acc: 0.9277


In [8]:
# Save the model
model2Conv.save('networkModels/FCN-1600-1600-128conv3-128conv3.h5')

In [23]:
# Use trained model to predict
predictAndSaveSet(testTrainPaths, modelBasicNN);

Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/XimeaT_20170613.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170531.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170622.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170802.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP20170622.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/XimeaT20170510.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/XimeaT20170622.mat
